In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/iADR')
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras import regularizers
from sklearn import preprocessing

Mounted at /content/drive


In [2]:
data1 = np.load('./data/gin_supervised_edgepred.npy')

y = pd.read_csv('./data/labels(2248).csv')
k_X_train,k_x_test,k_Y_train,k_y_test= train_test_split(data1,y,test_size=0.1,random_state=2021)

# model

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
import joblib

In [4]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
import joblib

In [5]:
m1 = keras.metrics.BinaryAccuracy(name='accuracy',threshold=0.5)
m2 = keras.metrics.Precision(name='precision')
m3 = keras.metrics.Recall(name='recall')
m4 = keras.metrics.AUC(name='AUC',multi_label=True,num_labels=27,num_thresholds=498)
m5 = keras.metrics.AUC(name='AUPR',curve='PR',multi_label=True,num_labels=27,num_thresholds=498)

In [5]:
#SVM
results1=[]
results2=[]
results3=[]
results4=[]
results5=[]
# 运行计时部分
time_start = time.time()
print(data1.shape)

kfolder = KFold(n_splits=10,shuffle=True,random_state=2021)
for i, (tra_id,val_id) in enumerate(kfolder.split(k_X_train,k_Y_train)):
  print(f"***********fold-{i+1}***********")

  c_x_train = k_X_train[tra_id]
  c_y_train = k_Y_train.iloc[tra_id]

  c_x_valid = k_X_train[val_id]
  c_y_valid = k_Y_train.iloc[val_id]



  # scaler = preprocessing.StandardScaler().fit(c_x_train)
  # c_x_train = scaler.transform(c_x_train)

  
  # scaler = preprocessing.StandardScaler().fit(c_x_valid)
  # c_x_valid = scaler.transform(c_x_valid)


  svc = SVC(probability=True)
  model = OneVsRestClassifier(svc, n_jobs=1)
  model.fit(c_x_train, c_y_train)



  y_pre = model.predict_proba(c_x_valid)
  # nlabel = np.array(c_y_valid)
  # plabel = np.where(c_pre_valid >= 0.5,1,0)    
  # print(nlabel,nlabel.shape)
  # print(y_pre_valid)
  # print(plabel)

  # y_pre = model.predict(c_x_valid)
  # np.save('./data/masking_RF_pre'+str(i+1)+'.npy',y_pre)



  m1.update_state(c_y_valid,y_pre)
  m2.update_state(c_y_valid,y_pre)
  m3.update_state(c_y_valid,y_pre)
  m4.update_state(c_y_valid,y_pre)
  m5.update_state(c_y_valid,y_pre)


  results1.append(m1.result().numpy())
  results2.append(m2.result().numpy())
  results3.append(m3.result().numpy())
  results4.append(m4.result().numpy())
  results5.append(m5.result().numpy())


  print(f"{i+1}-fold cv val Accuracy score: {m1.result().numpy()}")
  print(f"{i+1}-fold cv val Precision score: {m2.result().numpy()}")
  print(f"{i+1}-fold cv val Recall score: {m3.result().numpy()}")
  print(f"{i+1}-fold cv val AUC score: {m4.result().numpy()}")
  print(f"{i+1}-fold cv val AUPR score: {m5.result().numpy()}")
  print("\n")


  # 6、保存模型
  joblib.dump(model,'./model/edeg/SVM_edeg_10fold'+str(i+1)+'.pkl')





print("******************************************************************")
# print(np.array(results))

print(f"-Accuracy score_mean:{np.mean(results1)}")
print(f"-Accuracy score_std:{np.std(results1)}")

print(f"-Precision score_mean:{np.mean(results2)}")
print(f"-Precision score_std:{np.std(results2)}")

print(f"-Recall score_mean:{np.mean(results3)}")
print(f"-Recall score_std:{np.std(results3)}")

print(f"-AUC score_mean:{np.mean(results4)}")
print(f"-AUC score_std:{np.std(results4)}")

print(f"-AUPR score_mean:{np.mean(results5)}")
print(f"-AUPR score_std:{np.std(results5)}")






    
# 运行计时部分
time_end = time.time()
print(f"total running time: {(time_end - time_start)/60} minites")

(2248, 300)
***********fold-1***********
1-fold cv val Accuracy score: 0.7980296611785889
1-fold cv val Precision score: 0.7998361587524414
1-fold cv val Recall score: 0.8867695927619934
1-fold cv val AUC score: 0.7043075561523438
1-fold cv val AUPR score: 0.7263333797454834


***********fold-2***********
2-fold cv val Accuracy score: 0.7934683561325073
2-fold cv val Precision score: 0.7840248942375183
2-fold cv val Recall score: 0.8970156311988831
2-fold cv val AUC score: 0.7039100527763367
2-fold cv val AUPR score: 0.7135056853294373


***********fold-3***********
3-fold cv val Accuracy score: 0.7952320575714111
3-fold cv val Precision score: 0.7929947972297668
3-fold cv val Recall score: 0.891848087310791
3-fold cv val AUC score: 0.695745587348938
3-fold cv val AUPR score: 0.7124255895614624


***********fold-4***********
4-fold cv val Accuracy score: 0.7959994077682495
4-fold cv val Precision score: 0.7991828322410583
4-fold cv val Recall score: 0.8854685425758362
4-fold cv val AUC

In [6]:
# 7、模型的加载使用
model2 = joblib.load('./model/edeg/SVM_edeg_10fold1.pkl')
history  = model2.fit(k_X_train,k_Y_train)
y_pre = model2.predict_proba(k_x_test)
np.save('./model/edeg/edeg_svm_pre.npy',y_pre)

m1.update_state(k_y_test,y_pre)
m2.update_state(k_y_test,y_pre)
m3.update_state(k_y_test,y_pre)
m4.update_state(k_y_test,y_pre)
m5.update_state(k_y_test,y_pre)



print(f"Accuracy score: {m1.result().numpy()}")
print(f"Precision score: {m2.result().numpy()}")
print(f"Recall score: {m3.result().numpy()}")
print(f"AUC score: {m4.result().numpy()}")
print(f"AUPR score: {m5.result().numpy()}")

Accuracy score: 0.8128395080566406
Precision score: 0.8334953784942627
Recall score: 0.8835351467132568
AUC score: 0.7056541442871094
AUPR score: 0.7698445916175842


In [5]:
results1=[]
results2=[]
results3=[]
results4=[]
results5=[]
# 运行计时部分
time_start = time.time()
print(data1.shape)

kfolder = KFold(n_splits=10,shuffle=True,random_state=2021)
for i, (tra_id,val_id) in enumerate(kfolder.split(k_X_train,k_Y_train)):
  print(f"***********fold-{i+1}***********")

  c_x_train = k_X_train[tra_id]
  c_y_train = k_Y_train.iloc[tra_id]

  c_x_valid = k_X_train[val_id]
  c_y_valid = k_Y_train.iloc[val_id]



  # scaler = preprocessing.StandardScaler().fit(c_x_train)
  # c_x_train = scaler.transform(c_x_train)

  
  # scaler = preprocessing.StandardScaler().fit(c_x_valid)
  # c_x_valid = scaler.transform(c_x_valid)



  rf = RandomForestClassifier(n_estimators=100)
  model = OneVsRestClassifier(rf, n_jobs=-1)
  history = model.fit(c_x_train, c_y_train)



  y_pre = model.predict_proba(c_x_valid)
  # nlabel = np.array(c_y_valid)
  # plabel = np.where(c_pre_valid >= 0.5,1,0)    
  # print(nlabel,nlabel.shape)
  # print(y_pre_valid)
  # print(plabel)

  # y_pre = model.predict(c_x_valid)
  # np.save('./data/masking_RF_pre'+str(i+1)+'.npy',y_pre)



  m1.update_state(c_y_valid,y_pre)
  m2.update_state(c_y_valid,y_pre)
  m3.update_state(c_y_valid,y_pre)
  m4.update_state(c_y_valid,y_pre)
  m5.update_state(c_y_valid,y_pre)


  results1.append(m1.result().numpy())
  results2.append(m2.result().numpy())
  results3.append(m3.result().numpy())
  results4.append(m4.result().numpy())
  results5.append(m5.result().numpy())


  print(f"{i+1}-fold cv val Accuracy score: {m1.result().numpy()}")
  print(f"{i+1}-fold cv val Precision score: {m2.result().numpy()}")
  print(f"{i+1}-fold cv val Recall score: {m3.result().numpy()}")
  print(f"{i+1}-fold cv val AUC score: {m4.result().numpy()}")
  print(f"{i+1}-fold cv val AUPR score: {m5.result().numpy()}")
  print("\n")


  # 6、保存模型
  joblib.dump(model,'./model/edeg/RF_edeg_10fold'+str(i+1)+'.pkl')





print("******************************************************************")
# print(np.array(results))

print(f"-Accuracy score_mean:{np.mean(results1)}")
print(f"-Accuracy score_std:{np.std(results1)}")

print(f"-Precision score_mean:{np.mean(results2)}")
print(f"-Precision score_std:{np.std(results2)}")

print(f"-Recall score_mean:{np.mean(results3)}")
print(f"-Recall score_std:{np.std(results3)}")

print(f"-AUC score_mean:{np.mean(results4)}")
print(f"-AUC score_std:{np.std(results4)}")

print(f"-AUPR score_mean:{np.mean(results5)}")
print(f"-AUPR score_std:{np.std(results5)}")






    
# 运行计时部分
time_end = time.time()
print(f"total running time: {(time_end - time_start)/60} minites")

(2248, 300)
***********fold-1***********
1-fold cv val Accuracy score: 0.8016785383224487
1-fold cv val Precision score: 0.8122886419296265
1-fold cv val Recall score: 0.8725401163101196
1-fold cv val AUC score: 0.7103752493858337
1-fold cv val AUPR score: 0.7450223565101624


***********fold-2***********
2-fold cv val Accuracy score: 0.7982120513916016
2-fold cv val Precision score: 0.7964330315589905
2-fold cv val Recall score: 0.8838719725608826
2-fold cv val AUC score: 0.7120762467384338
2-fold cv val AUPR score: 0.7291814088821411


***********fold-3***********
3-fold cv val Accuracy score: 0.7965092062950134
3-fold cv val Precision score: 0.8016261458396912
3-fold cv val Recall score: 0.8785822987556458
3-fold cv val AUC score: 0.7058837413787842
3-fold cv val AUPR score: 0.725750207901001


***********fold-4***********
4-fold cv val Accuracy score: 0.7970041632652283
4-fold cv val Precision score: 0.8075124025344849
4-fold cv val Recall score: 0.8726422190666199
4-fold cv val AU

In [1]:
# 7、模型的加载使用
model2 = joblib.load('./model/edeg/RF_edeg_10fold1.pkl')
history  = model2.fit(k_X_train,k_Y_train)
y_pre = model2.predict_proba(k_x_test)
np.save('./model/edeg/edeg_RF_pre.npy',y_pre)

m1.update_state(k_y_test,y_pre)
m2.update_state(k_y_test,y_pre)
m3.update_state(k_y_test,y_pre)
m4.update_state(k_y_test,y_pre)
m5.update_state(k_y_test,y_pre)



print(f"Accuracy score: {m1.result().numpy()}")
print(f"Precision score: {m2.result().numpy()}")
print(f"Recall score: {m3.result().numpy()}")
print(f"AUC score: {m4.result().numpy()}")
print(f"AUPR score: {m5.result().numpy()}")

NameError: ignored

# deepDDI

In [3]:
METRICS = [
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.AUC(name='auc',multi_label=True,num_labels=27,num_thresholds=498),
      keras.metrics.AUC(name='prc',curve='PR',multi_label=True,num_labels=27,num_thresholds=498), # precision-recall curve
]

def create_model_DDI(metrics=METRICS):
  model = Sequential([
    layers.Flatten(name="my_intermediate_layer"),
    layers.Dropout(0.5),
    layers.Dense(2048),
    layers.ReLU(),
    layers.Dropout(0.5),
    layers.Dense(2048),
    layers.ReLU(),
    layers.Dropout(0.5),
    layers.Dense(2048),
    layers.ReLU(),
    layers.Dropout(0.5),
    layers.Dense(2048),
    layers.ReLU(),
    layers.Dropout(0.5),
    layers.Dense(2048),
    layers.ReLU(),
    layers.Dropout(0.5),
    layers.Dense(2048),
    layers.ReLU(),
    layers.Dropout(0.5),
    layers.Dense(2048),
    layers.ReLU(),
    layers.Dropout(0.5),
    layers.Dense(2048),
    layers.ReLU(),
    layers.Dropout(0.5),
    layers.Dense(27, activation='sigmoid', name="my_last_layer"),
    ])
  return model

In [4]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau#回调函数
# # 定义回调函数：保存最优模型                     
checkpoint = ModelCheckpoint("./data/fp_cross-model.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             save_weights_only=False,
                             verbose=1)
# 定义回调函数：提前终止训练
earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0.0001, 
                          patience = 30,
                          verbose = 3,
                          mode = 'min',
                          restore_best_weights = True)
# 定义回调函数：学习率衰减
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.2,
                              patience = 5,
                              verbose = 1,
                              min_delta = 0.0001)

# 将回调函数组织为回调列表
callbacks = [earlystop,reduce_lr]

In [5]:
results=[]
# 运行计时部分
time_start = time.time()
print(data1.shape)

kfolder = KFold(n_splits=10,shuffle=True,random_state=2021)
for i, (tra_id,val_id) in enumerate(kfolder.split(k_X_train,k_Y_train)):
  print(f"***********fold-{i+1}***********")

  c_x_train = k_X_train[tra_id]
  c_y_train = k_Y_train.iloc[tra_id]

  c_x_valid = k_X_train[val_id]
  c_y_valid = k_Y_train.iloc[val_id]



  model = create_model_DDI()





  opt = Adam(learning_rate=0.001)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=METRICS)
    
  # training
  history = model.fit(c_x_train,c_y_train,epochs=300,batch_size=128,
                      validation_data=(c_x_valid, c_y_valid.values),callbacks=callbacks,)
  


  y_pre = model.predict(c_x_valid)
  # y_pre1 = model.predict(k_x_test)
  # np.save('./data/FP_DDI__pre'+str(i+1)+'.npy',y_pre)

  baseline_results = model.evaluate(c_x_valid, c_y_valid.values, verbose=0)
  for name, value in zip(model.metrics_names, baseline_results):
    print(name, ': ', value)
  print()


  model.save_weights('./model/edeg/DDI_edeg_10fold'+str(i+1)+'.h5')

  result = np.array(baseline_results)
  results.append(result)



print("******************************************************************")
# print(np.array(results))

print(f"-Accuracy score_mean:{np.mean(np.array(results)[:,[1]])}")
# print(f"-Accuracy score_mean:{np.array(results)[:,[1]]}")
print(f"-Accuracy score_mean:{np.std(np.array(results)[:,[1]])}")

print(f"-Precision score_mean:{np.mean(np.array(results)[:,[2]])}")
print(f"-Precision score_mean:{np.std(np.array(results)[:,[2]])}")

print(f"-Recall score_mean:{np.mean(np.array(results)[:,[3]])}")
print(f"-Recall score_mean:{np.std(np.array(results)[:,[3]])}")

print(f"-AUC score_mean:{np.mean(np.array(results)[:,[4]])}")
print(f"-AUC score_mean:{np.std(np.array(results)[:,[4]])}")

print(f"-AUPR score_mean:{np.mean(np.array(results)[:,[5]])}")
print(f"-AUPR score_mean:{np.std(np.array(results)[:,[5]])}")






    
# 运行计时部分
time_end = time.time()
print(f"total running time: {(time_end - time_start)/60} minites")

(2248, 300)
***********fold-1***********
Epoch 1/300
15/15 [==============================] - 16s 814ms/step - loss: 0.6243 - precision: 0.7406 - recall: 0.7729 - accuracy: 0.6970 - auc: 0.5094 - prc: 0.6110 - val_loss: 0.5082 - val_precision: 0.7707 - val_recall: 0.8526 - val_accuracy: 0.7583 - val_auc: 0.5404 - val_prc: 0.6123 - lr: 0.0010
Epoch 2/300
15/15 [==============================] - 11s 737ms/step - loss: 0.5246 - precision: 0.7663 - recall: 0.8617 - accuracy: 0.7559 - auc: 0.4983 - prc: 0.6085 - val_loss: 0.5071 - val_precision: 0.7429 - val_recall: 0.9131 - val_accuracy: 0.7572 - val_auc: 0.5936 - val_prc: 0.6407 - lr: 0.0010
Epoch 3/300
15/15 [==============================] - 11s 725ms/step - loss: 0.5101 - precision: 0.7580 - recall: 0.8800 - accuracy: 0.7560 - auc: 0.5187 - prc: 0.6168 - val_loss: 0.5109 - val_precision: 0.7565 - val_recall: 0.8834 - val_accuracy: 0.7584 - val_auc: 0.6296 - val_prc: 0.6553 - lr: 0.0010
Epoch 4/300
15/15 [==============================]

In [5]:
#加载模型
from keras.models import load_model
# model.load_weights(‘model_weights.h5')
model2 = create_model_DDI()

model2.build(k_X_train.shape)#加这一步有了解决

model2.load_weights('./model/edeg/DDI_edeg_10fold5.h5')

opt = Adam(learning_rate=0.001)
model2.compile(loss='binary_crossentropy', optimizer=opt, metrics=METRICS)
model2.fit(k_X_train,k_Y_train,batch_size=128, epochs=300,validation_data=(k_x_test,k_y_test),callbacks=[earlystop])
model2.evaluate(k_x_test,k_y_test,return_dict=True)
y_pre = model2.predict(k_x_test)
np.save('./model/edeg/fp_edeg_pre.npy',y_pre)

Epoch 1/300
16/16 [==============================] - 15s 767ms/step - loss: 0.4635 - precision: 0.7901 - recall: 0.8706 - accuracy: 0.7807 - auc: 0.6887 - prc: 0.7074 - val_loss: 0.4483 - val_precision: 0.8246 - val_recall: 0.8565 - val_accuracy: 0.7919 - val_auc: 0.6954 - val_prc: 0.7193
Epoch 2/300
16/16 [==============================] - 11s 694ms/step - loss: 0.4610 - precision: 0.7992 - recall: 0.8622 - accuracy: 0.7845 - auc: 0.6908 - prc: 0.7082 - val_loss: 0.4457 - val_precision: 0.8221 - val_recall: 0.8606 - val_accuracy: 0.7919 - val_auc: 0.6984 - val_prc: 0.7241
Epoch 3/300
16/16 [==============================] - 11s 693ms/step - loss: 0.4581 - precision: 0.8010 - recall: 0.8621 - accuracy: 0.7859 - auc: 0.6974 - prc: 0.7134 - val_loss: 0.4414 - val_precision: 0.8160 - val_recall: 0.8833 - val_accuracy: 0.7982 - val_auc: 0.7079 - val_prc: 0.7264
Epoch 4/300
16/16 [==============================] - 11s 709ms/step - loss: 0.4566 - precision: 0.7963 - recall: 0.8726 - accuracy